In [2]:
pip install msedge-selenium-tools selenium==3.141

     |████████████████████████████████| 904 kB 133 kB/s eta 0:00:01
  Attempting uninstall: selenium
    Found existing installation: selenium 4.1.0
    Uninstalling selenium-4.1.0:
      Successfully uninstalled selenium-4.1.0
Note: you may need to restart the kernel to use updated packages.


In [132]:
import csv
from datetime import datetime
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [133]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [134]:
def save_data_to_file(records):
    """Save data to csv file"""
    with open("results1.csv", 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'JobUrl'])
        writer.writerows(records)

In [135]:
def get_record(card):
    """Extract job data from single card"""
    job_title = card.find_element_by_class_name('jobTitle').text
    company = card.find_element_by_class_name('companyName').text
    #location = card.find_element_by_class_name('location').text
    post_date = card.find_element_by_class_name('date').text
    extract_date = datetime.today().strftime('%Y-%m-%d')
    #summary = card.find_element_by_class_name('summary').text
    job_url = card.find_element_by_class_name('jobTitle').get_attribute('href')
    return (job_title, company, location, post_date, extract_date, summary, job_url)

In [139]:
def get_page_records(cards, job_list, url_set):
    """Extract all cards from the page"""
    for card in cards:
        record = get_record(card)
        # add if job title exists and not duplicate
        if record[0] and record[-1] not in url_set:
            job_list.append(record)
            url_set.add(record[-1])

In [140]:
def main(position, location):
    """Run the main program routine"""
    scraped_jobs = []
    scraped_urls = set()
    
    url = get_url(position, location)
    
    # setup web driver
    #DRIVER_PATH = '/Users/a.groon/chromedriver'
    driver =webdriver.Chrome(ChromeDriverManager().install())
    #options = EdgeOptions()
    #options.use_chromium = True
    #driver = Edge(options=options)
    driver.implicitly_wait(5)
    driver.get(url)        
    
    # extract the job data
    while True:
        cards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
        get_page_records(cards, scraped_jobs, scraped_urls)
        try:
            driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            driver.find_element_by_id('popover-x').click()  # to handle job notification popup
            get_page_records(cards, scraped_jobs, scraped_urls)
            continue
    
    # close driver and save records
    driver.quit()
    save_data_to_file(scraped_jobs)

In [141]:
# run a search
main('data scientist', 'chicago, il')



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/a.groon/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
